# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Nome**: João Vitor Averaldo Antunes

**RA**: 813979


---
### Packages utilizados ao longo do experimento e considerações adicionais
* Todos pertencem a distribuição Anaconda
* O código foi testado e desenvolvido em Python 3.10.11
* Para o correto funcionamento do código é importante que a base de dados, o conjunto de treinamento e o conjunto de testes sejam inseridos com o nome de RHP_data.csv, train.csv e test.csv, respectivamente, dentro da pasta "/data".

In [ ]:
# Packages utilizados ao longo do desenvolvimento
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install numpy
%pip install scikit-learn

---
### Imports iniciais
Imports das bibliotecas e funções que serão utilizadas nas etapas de análise exploratória, pré-processamento, execução dos experimentos e análise dos resultados.

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Analise exploratoria
from scripts.analise_exploratoria import (
    carregar_dados,
    construir_df_treino,
    construir_df_teste,
    plotar_histogramas,
    plotar_correlacao,
    plotar_boxplots,
    analisar_variaveis_categoricas
)

# Preprocessamento
from scripts.preprocessamento import preprocessar_df

# Experimentos
from scripts.experimentos import executar_experimentos

# Analise de resultados
from scripts.analise_resultados import (
    analisar_resultados,
    plot_learning_curve,
    plot_pca
)

print ("\nImports realizados com sucesso.")

---
### Análise exploratória

Carregamento dos dados e construção dos dataframes de treino e teste. Em seguida, são realizadas diversas análises exploratórias, como visualização das primeiras linhas, histogramas, matriz de correlação, boxplots e análise de variáveis categóricas.

Explicação dos atributos:

* <b>CLASSE:</b> Normal (potencialmente sem patologia) ou Anormal (potencialmente com patologia).

* <b>PESO:</b> Medida essencial para avaliar o crescimento, desnutrição ou obesidade.

* <b>ALTURA:</b> Indicador da estatura física.

* <b>IMC:</b> Relação entre peso e altura que ajuda a identificar o estado nutricional.

* <b>ATENDIMENTO:</b> Registra as datas das consultas.

* <b>DN:</b> Data de nascimento, útil para calcular a idade e ajustar parâmetros de crescimento.

* <b>IDADE:</b> Determina a fase do desenvolvimento do paciente.

* <b>CONVENIO:</b> Indica o plano de saúde do paciente.

* <b>PULSOS:</b> Avaliação dos pulsos, importante para identificar problemas circulatórios.

* <b>PA SISTÓLICA:</b> Pressão arterial durante a contração do coração.

* <b>PA DIASTÓLICA:</b> Pressão arterial no relaxamento do coração.

* <b>PPA:</b> Diferença entre a pressão sistólica e diastólica.

* <b>B2:</b> Características do segundo som cardíaco.

* <b>SOPRO:</b> Presença de ruídos cardíacos que pode sinalizar anomalias.

* <b>FC:</b> Frequência cardíaca, utilizada para monitorar o ritmo e detectar arritmias.

* <b>HDA1:</b> Primeira parte da história clínica, descrevendo o início dos sintomas.

* <b>HDA2:</b> Continuação da história, detalhando a evolução.

* <b>SEXO:</b> Gênero do paciente, relevante para identificar diferentes manifestações de doenças.

* <b>MOTIVO1:</b> Motivo principal que levou à consulta.

* <b>MOTIVO2:</b> Possível outro motivo complementar que levou à consulta.



In [ ]:
# Carregamento dos dados
rhp_data, train_labels, test_ids = carregar_dados(
    rhp_data_path="data/RHP_data.csv",
    train_path="data/train.csv",
    test_path="data/test.csv"
)

# Construcao do df_treino e df_teste
df_treino = construir_df_treino(rhp_data, train_labels)
df_teste = construir_df_teste(rhp_data, test_ids)

# Visulizacao das primeiras linhas do df_treino
display(df_treino.head(10))

# Plot dos histogramas das colunas numericas
plotar_histogramas(df_treino)

# Plot da matriz de correlacao das colunas numericas
plotar_correlacao(df_treino)

# Plot dos boxplots para detectar outliers
plotar_boxplots(df_treino)

# Analise das variaveis categoricas
analisar_variaveis_categoricas(df_treino)

print("\nAnálise exploratória completa com sucesso.")

---
### Pré-processamento

Aplicado o pré-processamento tanto no dataframe de treino quanto no de teste. Em seguida, novas visualizações são geradas para confirmar as transformações realizadas.

In [ ]:
# Para o df_treino:
df_treino_proc = preprocessar_df(df_treino.copy())

# Para o df_teste:
df_teste_proc = preprocessar_df(df_teste.copy())

# Visualizacao das primeiras linhas do df_treino
display(df_treino_proc.head(10))

# Plot dos histogramas das colunas numericas
plotar_histogramas(df_treino_proc)

# Plot da matriz de correlacao das colunas numericas
plotar_correlacao(df_treino_proc)

# Plot dos boxplots para analise dos outlier
plotar_boxplots(df_treino_proc)

print("\nPré-processamento realizado com sucesso.")

---
### Experimento

Execução dos experimentos de machine learning e armazenamento dos resultados em um DataFrame para facilitar a análise. São gerados gráficos de avaliação, selecionado o melhor modelo com base na métrica ROC-AUC e, por fim, um arquivo de submissão é criado com as previsões.

In [ ]:
# Execucao dos experimentos
resultados = executar_experimentos(df_treino_proc, target_col='CLASSE')

# Salva os resultados em um DataFrame para facilitar a analise
resultados_df = pd.DataFrame({
    modelo: metrics['Resultados'] for modelo, metrics in resultados.items()
}).T

display(resultados_df)

# Visualizacao dos resultados
# Define as metricas a serem plotadas
metricas = ['Acuracia', 'Precisao', 'Recall', 'F1-Score', 'ROC-AUC']

plt.figure(figsize=(20, 10))
for i, metrica in enumerate(metricas, 1):
    plt.subplot(2, 3, i)
    sns.barplot(x=resultados_df.index, y=resultados_df[metrica], palette="viridis", hue=resultados_df.index, legend=False)
    plt.title(metrica)
    plt.ylim(0, 1)
    plt.ylabel(metrica)
    plt.xlabel('Modelo')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Geracao do arquivo de submissao

# Seleciona o melhor modelo baseado na ROC-AUC
melhor_modelo_nome = resultados_df['ROC-AUC'].idxmax()
melhor_modelo = resultados[melhor_modelo_nome]['Modelo']
print(f"\nMelhor modelo: {melhor_modelo_nome}")

# Verifica a coluna de identificador no conjunto de teste
id_col = 'Id'

# Alinha as colunas do teste com as do treino
colunas_treino = df_treino_proc.drop(columns=['CLASSE']).columns
for col in colunas_treino:
    if col not in df_teste_proc.columns:
        df_teste_proc[col] = 0
df_teste_proc = df_teste_proc[colunas_treino]

# Faz previsoes de probabilidade usando o melhor modelo
probabilidades = melhor_modelo.predict_proba(df_teste_proc)[:,1]

# Cria o DataFrame de submissao
submissao = pd.DataFrame({
    id_col: df_teste[id_col],
    'Predicted': probabilidades
})
submissao['Predicted'] = submissao['Predicted'].clip(0, 1)
display(submissao.head())

# Salva a submissao em um arquivo CSV
submissao.to_csv('submissao.csv', index=False)
print("\nGeração do arquivo realizada com sucesso.")

---
### Análise dos Resultados

Análise dos resultados obtidos, incluindo a plotagem da curva de aprendizado do melhor modelo e a aplicação da redução de dimensionalidade utilizando PCA para melhor compreensão dos dados.

In [ ]:
# Analise dos resultados
analisar_resultados(resultados, resultados_df)

# Curva de Aprendizado para o melhor modelo
X_train = df_treino_proc.drop(columns=['CLASSE'])
y_train = df_treino_proc['CLASSE']
print("\nCurva de Aprendizado para o melhor modelo:")
plot_learning_curve(melhor_modelo, X_train, y_train, title=f"Curva de Aprendizado - {melhor_modelo_nome}")

# Analise de reducao de dimensionalidade usando PCA
print("\nAnálise de Redução de Dimensionalidade com PCA:")
plot_pca(X_train, y_train)

print("\nAnálise dos resultados completa com sucesso.")